<a href="https://colab.research.google.com/github/keijikk/ml_study/blob/master/03_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!apt-get install lshw
#!lshw

In [18]:
import os
import numpy as np
import tensorflow as tf
from time import gmtime, strftime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.python.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from tensorflow.keras.layers import Conv2D, MaxPooling2D


from google.colab import drive
drive.mount('/content/drive')
tb_home_dir = '/content/drive/My Drive/ML/直感 Deep Learning/drive/'


! npm install -g localtunnel
!rm url.txt

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(tb_home_dir)
)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/client
+ localtunnel@1.9.1
updated 1 package in 1.753s


   ╭─────────────────────────────────────╮
   │                                     │
   │   Update available 5.7.1 → 6.4.1    │
   │     Run npm i -g npm to update      │
   │                                     │
   ╰─────────────────────────────────────╯



In [5]:
!cat url.txt

your url is: https://unlucky-husky-54.localtunnel.me


In [0]:
def lenet(input_shape, num_classes):
  model = Sequential()
  model.add(Conv2D(filters=20, kernel_size=5, padding='same', input_shape=input_shape, activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(filters=50, kernel_size=5, padding='same', input_shape=input_shape, activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(500, activation="relu"))
  model.add(Dense(num_classes))
  model.add(Activation("softmax"))
  return model
  

In [0]:
class MNISTDataset():
  def __init__(self):
    self.image_shape=(28, 28, 1) # grayscale
    self.num_classes = 10
    
  def get_batch(self):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
    y_train, y_test = [self.preprocess(d, True) for d in [y_train, y_test]]
    return x_train, y_train, x_test, y_test
    
  def preprocess(self, data, label_data=False):
    if label_data: # y label
      data = utils.to_categorical(data, self.num_classes)
    else:
      data = data.astype("float32")
      data /= 255
      shape = (data.shape[0], ) + self.image_shape # data numの後ろに image shapeをつける
      data = data.reshape(shape)
    return data

In [0]:
class Trainer():
  def __init__(self, model, loss, optimizer, set_dir_name):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    self.verbose=1
    self.log_dir = self.get_log_dir(set_dir_name=set_dir_name)

    
  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)
    
    self._target.fit(
        x_train, y_train,
        batch_size=batch_size, epochs=epochs,
        validation_split=validation_split,
        callbacks=[TensorBoard(log_dir=self.log_dir)],
        verbose=self.verbose
    )
  
  
  def get_log_dir(self, set_dir_name = ''):
    """return log_dir"""
    tictoc = strftime('%a_%d_%b_%Y_%H_%M_%S', gmtime())
    directory_name = tictoc
    log_dir = set_dir_name +'_' + directory_name
    os.mkdir(log_dir)
    return log_dir


# MNIST

In [9]:
dataset = MNISTDataset()
model = lenet(dataset.image_shape, dataset.num_classes)
model.summary()

x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam(), set_dir_name="conv_v1")

trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Train loss:", score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2450)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               1225500   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5010      
__________

# CIFAR-10

In [0]:
from tensorflow.keras.datasets import cifar10
class CIFAR10Dataset():
  def __init__(self):
    self.image_shape=(32, 32, 3) # grayscale
    self.num_classes = 10
    
  def get_batch(self):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
    y_train, y_test = [self.preprocess(d, True) for d in [y_train, y_test]]
    return x_train, y_train, x_test, y_test
    
  def preprocess(self, data, label_data=False):
    if label_data: # y label
      data = utils.to_categorical(data, self.num_classes)
    else:
      data = data.astype("float32")
      data /= 255
      shape = (data.shape[0], ) + self.image_shape # data numの後ろに image shapeをつける
      data = data.reshape(shape)
    return data
  
class Trainer():
  def __init__(self, model, loss, optimizer, set_dir_name):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    self.verbose=1
    self.log_dir = self.get_log_dir(set_dir_name=set_dir_name)
    self.model_file_name = "model_file.hdf5"


  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)

    self._target.fit(
        x_train, y_train,
        batch_size=batch_size, epochs=epochs,
        validation_split=validation_split,
        callbacks=[TensorBoard(log_dir=self.log_dir),
                  ModelCheckpoint(os.path.join(self.log_dir, self.model_file_name), save_best_only=True)],
        verbose=self.verbose
    )


  def get_log_dir(self, set_dir_name = ''):
    """return log_dir"""
    tictoc = strftime('%a_%d_%b_%Y_%H_%M_%S', gmtime())
    directory_name = tictoc
    log_dir = set_dir_name +'_' + directory_name
    os.mkdir(log_dir)
    return log_dir


In [21]:
dataset = CIFAR10Dataset()
model = lenet(dataset.image_shape, dataset.num_classes)
model.summary()

x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam(), set_dir_name="conv_v2")

trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Train loss:", score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 20)        1520      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 50)        25050     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 8, 8, 50)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               1600500   
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5010      
__________

In [0]:
def network(input_shape, num_classes):
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=3, padding='same', input_shape=input_shape, activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(filters=64, kernel_size=5, padding='same', input_shape=input_shape, activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(filters=64, kernel_size=5, input_shape=input_shape, activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(512, activation="relu"))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation("softmax"))
  return model
  

In [24]:
dataset = CIFAR10Dataset()
model = network(dataset.image_shape, dataset.num_classes)
model.summary()

x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam(), set_dir_name="conv_v2")

trainer.train(x_train, y_train, batch_size=128, epochs=12, validation_split=0.2)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Train loss:", score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)        51264     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 4, 4, 64)          102464    
__________

# Data Augmentation

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [0]:

class Trainer():
  def __init__(self, model, loss, optimizer, set_dir_name):
    self._target = model
    self._target.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    self.verbose=1
    self.log_dir = self.get_log_dir(set_dir_name=set_dir_name)
    self.model_file_name = "model_file.hdf5"


  def train(self, x_train, y_train, batch_size, epochs, validation_split):
    if os.path.exists(self.log_dir):
      import shutil
      shutil.rmtree(self.log_dir)
    os.mkdir(self.log_dir)
    
    datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False, 
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False)
    
    print(x_train.shape)
    datagen.fit(x_train)
    print(x_train.shape)
    
    indices = np.arange(x_train.shape[0]) # data num
    np.random.shuffle(indices)
    validation_size = int(x_train.shape[0]*validation_split)
    x_train, x_valid = x_train[indices[:-validation_size],:], x_train[indices[-validation_size:],:]
    y_train, y_valid = y_train[indices[:-validation_size],:], y_train[indices[-validation_size:],:]
    

    print(self.log_dir)
    print(os.path.join(self.log_dir, self.model_file_name))
    self._target.fit_generator(
        datagen.flow(x_train, y_train, batch_size=batch_size),
        steps_per_epoch=x_train.shape[0] // batch_size,
        epochs=epochs,
        validation_data=(x_valid, y_valid),
        callbacks=[TensorBoard(log_dir=self.log_dir),
                  ModelCheckpoint(os.path.join(self.log_dir, self.model_file_name), save_best_only=True)],
        verbose=self.verbose,
        workers=4
        
    )

  def get_log_dir(self, set_dir_name = ''):
    """return log_dir"""
    tictoc = strftime('%a_%d_%b_%Y_%H_%M_%S', gmtime())
    directory_name = tictoc
    log_dir = os.path.join(tb_home_dir, set_dir_name +'_' + directory_name)
    os.mkdir(log_dir)
    return log_dir


In [42]:
dataset = CIFAR10Dataset()
model = network(dataset.image_shape, dataset.num_classes)
#model.summary()

x_train, y_train, x_test, y_test = dataset.get_batch()
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam(), set_dir_name="conv_v2")

trainer.train(x_train, y_train, batch_size=128, epochs=3, validation_split=0.2)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Train loss:", score[1])


(50000, 32, 32, 3)
(50000, 32, 32, 3)
/content/drive/My Drive/ML/直感 Deep Learning/drive/conv_v2_Sat_03_Nov_2018_11_15_05
/content/drive/My Drive/ML/直感 Deep Learning/drive/conv_v2_Sat_03_Nov_2018_11_15_05/model_file.hdf5
Epoch 1/3
312/312 [==============================] - 31s 98ms/step - loss: 1.8475 - acc: 0.3095 - val_loss: 1.4662 - val_acc: 0.4610
Epoch 2/3
312/312 [==============================] - 29s 95ms/step - loss: 1.4624 - acc: 0.4632 - val_loss: 1.2516 - val_acc: 0.5442
Epoch 3/3
312/312 [==============================] - 29s 94ms/step - loss: 1.3401 - acc: 0.5167 - val_loss: 1.1183 - val_acc: 0.6007
Test loss: 1.1177152481079102
Train loss: 0.5981


# 再利用

In [45]:
from pathlib import Path
from PIL import Image
from tensorflow.keras.models import load_model
tb_home_dir = '/content/drive/My Drive/ML/直感 Deep Learning/drive/'

model_path = os.path.join(tb_home_dir, 'conv_v2','model_file.hdf5')

model = load_model(model_path)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 16, 16, 64)        51264     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 4, 4, 64)          102464    
__________